## Simulations in JAX

%pip install jax numpy

In [1]:
import jax
import jax.numpy as jnp
from jax import random, vmap, jit, lax
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import os
import glob
from pathlib import Path
from functools import partial

# Enable double precision
jax.config.update("jax_enable_x64", True)

# Define parameters
population_range = jnp.arange(15, 1501, 15)
sample_count = 8
dt = 0.2
tmax = 50
gene_num = 3
kappa_g = 0.0001
kappa_f = 0.1
cycle = 5
r = 1.0
f0 = 1.0
morphogen = 0.0185 # Morphogen concentration -- affecting gene expression
h = 2
d = 2
beta_s = 0.175
beta_l = 0.125
alpha_max = 0.95 # Parameters for cell interactions (adhesion)
alpha_min = 0.65 # Parameters for cell interactions (adhesion)

# Generate response matrices -- Quantifying interaction strengths between genes
# Hill function will use this matrix for computing regulation

response = jnp.zeros((gene_num, gene_num * 2, 2))
response = response.at[0, 0, 0].set(0.7)
response = response.at[1, 0, 0].set(0.1)
response = response.at[1, 2, 0].set(-0.5)
response = response.at[1, 3, 0].set(-0.3)
response = response.at[1, 0, 1].set(-0.1)
response = response.at[1, 1, 1].set(0.9)
response = response.at[2, 1, 0].set(-0.5)

leak = jnp.array([0.02, 0.0025, 0.0025])
degra = jnp.array([0.75, 0.50, 0.75])

# Create output directories
for i in range(sample_count):
    os.makedirs(f"OUTPUT{i+1}", exist_ok=True)

# Create noise data -- setting random seed for each OUTPUT directory

total_steps = int(tmax * cycle / dt)
for i in range(sample_count):
    noise_file = f"OUTPUT{i+1}/noise.npy"
    if not os.path.exists(noise_file):
        key = random.PRNGKey(i)
        cellnum = population_range[-1]
        noise = jax.random.normal(key, shape=(cellnum, 6, total_steps)) * kappa_g
        np.save(noise_file, np.array(noise))

### Prerequisite functions:

In [2]:
@jit  # Just-in-time compilation for enabling faster runtime. Note the 
# use of decorator, which takes as input a function and returns a wrapper,
# in this case an object which is compiled to machine code right before / at the time of execution.

## Compute pairwise gene expresion differences
def compute_gene_regulation(gene_expr):
    intensity = jnp.clip(gene_expr, 0, 1)
    cellnum = intensity.shape[0]

    ## Use of broadcasting instead of loops,
    ## The None index specifies pairwise operations
    diff = intensity[:, None, :] - intensity[None, :, :]
    distance = jnp.sqrt(jnp.sum(diff**2, axis=-1))
    distance = jnp.where(jnp.eye(cellnum).astype(bool), jnp.inf, distance)
    
    distance_d = jnp.where(  ### Calculate long range or short range distances
        jnp.isinf(distance) | (distance < 1e-12),
        jnp.zeros_like(distance),
        jnp.power(1.0 / distance, d) ## Power law scaling
    )
    
    morphogen_contribution = jnp.matmul(distance_d, intensity

    # Clip morphogen contributions between a range of 0 and 1
    morphogen_effect = jnp.clip(morphogen * morphogen_contribution, 0, 1)
    
    intensity_combined = jnp.concatenate([intensity, morphogen_effect], axis=1)
    intensity_T = jnp.transpose(intensity_combined)
    intensity_reshaped = jnp.reshape(intensity_T, (1, gene_num * 2, 1, cellnum))
    intensity_expanded = jnp.tile(intensity_reshaped, (gene_num, 1, 2, 1))
    
    response_expanded = jnp.tile(response[..., None], (1, 1, 1, cellnum)) # ... specifies all axes up to the current location
    

    response_abs = jnp.abs(response_expanded)
    response_pow = jnp.where(response_abs > 1e-12, jnp.power(response_abs, h), 0.0)
    intensity_pow = jnp.power(intensity_expanded, h)
    denominator = response_pow + intensity_pow
    hill_term = jnp.where(denominator > 1e-12, response_pow / denominator, 0.0)
    
    regulation = (1.0 +
                 jnp.sign(response_expanded) * (0.5 - hill_term) -
                 0.5 * jnp.sign(response_abs)) ## Gene regulation strength governed by Hill equation
    
    regulation = jnp.prod(regulation, axis=2)
    regulation = jnp.where(jnp.abs(regulation - 1.0) < 1e-12, 0.0, regulation)
    regulation = jnp.sum(regulation, axis=1)
    
    return jnp.transpose(regulation)

@jit
def update_state(state, noise_step):
    positions, gene_expr = state # Load the current state
    cellnum = positions.shape[0]
    
    diff = positions[:, None] - positions[None, :]
    distance = jnp.sqrt(jnp.sum(diff**2, axis=-1))
    distance = jnp.where(jnp.eye(cellnum).astype(bool), jnp.inf, distance)
    
    short_range = alpha_max - beta_s * jnp.ones_like(distance)
    short_range = jnp.clip(short_range, alpha_min, alpha_max)
    short_range = jnp.where(jnp.eye(cellnum).astype(bool), jnp.nan, short_range)
    
    interact = (~jnp.isnan(short_range)) & (distance < 2 * r)
    direction = jnp.where(
        distance[..., None] > 1e-12,
        diff / distance[..., None],
        jnp.zeros_like(diff)
    )
    force_mag = -f0 / (2 * r * short_range) * distance + f0
    forces = jnp.sum(jnp.where(interact[..., None], force_mag[..., None] * direction, 0.0), axis=1)
    
    new_pos = positions + forces * dt + noise_step[:, :3]
    new_pos = new_pos - jnp.mean(new_pos, axis=0)
    
    regulation = compute_gene_regulation(gene_expr)
    new_gene = gene_expr + dt * (regulation - gene_expr * degra + leak)
    new_gene = jnp.clip(new_gene, 0, 1)
    
    return new_pos, new_gene

def initialize_positions() # Note that this is dynamic, so cannot be JIT-compiled
    nnn = 15
    grid = jnp.mgrid[1:nnn+1, 1:nnn+1, 1:nnn+1]
    positions = jnp.stack([
        (grid[0] - (nnn+1)/2) * 2 * r,
        (grid[1] - (nnn+1)/2) * 2 * r,
        (grid[2] - (nnn+1)/2) * 2 * r
    ]).reshape(3, -1).T
    
    key = random.PRNGKey(0)
    positions = random.permutation(key, positions) ## Randomly initialize positions
    return positions[:population_range[-1]]

In [3]:
def create_static_visualization(sample_idx: int, final_state: np.ndarray):
    positions = final_state[:, :3]
    gene_expr = final_state[:, 3:6]
    
    # Create figure with proper spacing
    fig = plt.figure(figsize=(20, 7))  # Slightly taller for better visibility
    plt.subplots_adjust(left=0.05, right=0.92, bottom=0.15, top=0.88, wspace=0.3)
    
    for i in range(3):
        ax = fig.add_subplot(1, 3, i+1, projection='3d')
        scatter = ax.scatter(
            positions[:, 0], positions[:, 1], positions[:, 2],
            c=gene_expr[:, i],
            cmap='RdBu',
            s=100,  # Large points
            alpha=1.0,  # Fully opaque
            vmin=0,
            vmax=1,
            edgecolors='black',
            linewidth=0.5  # Thinner edges for better visibility
        )
        
        # Set view angle and distance
        ax.view_init(elev=20, azim=45)
        ax.dist = 10
        
        # Adjust axis labels and ticks
        ax.set_xlabel('X', labelpad=10)
        ax.set_ylabel('Y', labelpad=10)
        ax.set_zlabel('Z', labelpad=10)
        ax.set_title(f'Gene {i+1}', pad=20)
        
        # Set consistent axis limits
        ax.set_xlim([-20, 20])
        ax.set_ylim([-20, 20])
        ax.set_zlim([-10, 10])  # Flatter z-axis
        
        # Add gridlines
        ax.grid(True, linestyle='--', alpha=0.3)
        
        # Adjust tick parameters
        ax.tick_params(axis='both', labelsize=10, pad=5)
        
        # Add colorbar only to the last subplot
        if i == 2:
            cbar = plt.colorbar(scatter, ax=ax, format='%.1f', pad=0.1)
            cbar.set_label('Gene Expression', labelpad=10, size=12)
            cbar.set_ticks([0, 0.5, 1.0])
            cbar.ax.tick_params(labelsize=10)
        
        # Remove legend if it exists
        if ax.get_legend() is not None:
            ax.get_legend().remove()
    
    plt.suptitle(f'Sample {sample_idx+1} Final State', y=0.97, size=14)  # Moved up slightly
    save_path = f"OUTPUT{sample_idx+1}/visualization.png"
    plt.savefig(save_path, dpi=300, bbox_inches='tight', pad_inches=0.3)
    plt.close()
    return save_path

### Running simulation

In [4]:
# Process samples using JAX parallelization
devices = jax.devices()
print(f"Available JAX devices: {devices}")

# Process samples
results = []
for sample_idx in range(sample_count):
    # Load noise data
    noise = np.load(f"OUTPUT{sample_idx+1}/noise.npy")
    
    # Initialize state
    positions = initialize_positions()
    gene_expr = jnp.ones((positions.shape[0], gene_num))
    state = (positions, gene_expr)
    
    total_steps = int(tmax * cycle / dt)
    batch_size = 100
    
    def step_fn(state, noise_step):
        new_state = update_state(state, noise_step)
        return new_state, new_state
    
    for t in range(0, total_steps, batch_size):
        end_t = min(t + batch_size, total_steps)
        batch_noise = noise[:, :, t:end_t]

        ### Parallelize state update across time steps:
        states = lax.scan(
            lambda s, n: step_fn(s, n),
            state,
            jnp.moveaxis(batch_noise, 2, 0)
        )[1]
        
        state = jax.tree.map(lambda x: x[-1], states)
        
        if t % 100 == 0:
            positions, gene_expr = state
            state_combined = jnp.concatenate([positions, gene_expr], axis=1)
            np.save(f"OUTPUT{sample_idx+1}/state_{t}.npy", np.array(state_combined))
            
            print(f"Sample {sample_idx+1}, step {t}/{total_steps}:")
            print(f"Position range: [{jnp.min(positions):.3f}, {jnp.max(positions):.3f}]")
            print(f"Gene expression range: [{jnp.min(gene_expr):.3f}, {jnp.max(gene_expr):.3f}]")
    
    positions, gene_expr = state
    final_state = jnp.concatenate([positions, gene_expr], axis=1)
    np.save(f"OUTPUT{sample_idx+1}/final_state.npy", np.array(final_state))
    
    create_static_visualization(sample_idx, np.array(final_state))
    
    print(f"Sample {sample_idx+1} completed successfully")
    results.append(True)

Available JAX devices: [CpuDevice(id=0)]
Sample 1, step 0/1250:
Position range: [-14.006, 14.252]
Gene expression range: [0.270, 1.000]
Sample 1, step 100/1250:
Position range: [-14.007, 14.254]
Gene expression range: [0.270, 1.000]
Sample 1, step 200/1250:
Position range: [-14.007, 14.253]
Gene expression range: [0.136, 1.000]
Sample 1, step 300/1250:
Position range: [-14.008, 14.254]
Gene expression range: [0.029, 1.000]
Sample 1, step 400/1250:
Position range: [-14.007, 14.254]
Gene expression range: [0.029, 1.000]
Sample 1, step 500/1250:
Position range: [-14.007, 14.256]
Gene expression range: [0.029, 1.000]
Sample 1, step 600/1250:
Position range: [-14.008, 14.255]
Gene expression range: [0.029, 1.000]
Sample 1, step 700/1250:
Position range: [-14.009, 14.258]
Gene expression range: [0.029, 1.000]
Sample 1, step 800/1250:
Position range: [-14.010, 14.259]
Gene expression range: [0.029, 1.000]
Sample 1, step 900/1250:
Position range: [-14.009, 14.262]
Gene expression range: [0.029

2025-03-04 14:22:09,576 - INFO - Sample 1, step 500/1250:


2025-03-04 14:22:09,577 - INFO - Position range: [-18.582, 19.285]


2025-03-04 14:22:09,577 - INFO - Gene expression range: [0.029, 1.000]


2025-03-04 14:22:10,567 - INFO - Sample 1, step 600/1250:


...

2025-03-04 14:24:14,370 - INFO - Sample 8 completed successfully


2025-03-04 14:24:14,371 - INFO - Completed sample 8/8


2025-03-04 14:24:14,372 - INFO - 
Complete workflow executed successfully


2025-03-04 14:24:14,372 - INFO - Successfully processed 8/8 samples using JAX acceleration


2025-03-04 14:24:14,373 - INFO - Device utilization: TFRT_CPU_0
